In [1]:
import DynamicSky as ds

ds.register_mesh("/Users/robertcordingly/Dropbox (Maestral)/Research/SAAF-Master/SAAF/python_template/sky_mesh.json")

In [ ]:
def hello_world(request, inspector):
    return "Hello World!"

ds.run(hello_world, {}, "skyf_128_af-south-1")

In [4]:
def cpu_split(request, inspector):
    import threading
    import time
    
    payload = request
    depth = request['depth']
    breadth = request['breadth']
    sleep_time = request['sleep_time']
    url = request['url']
    final_url = request['final_url']
    payload['depth'] = depth - 1
    results = []
    
    inspector.addAttribute("cpuInfo", inspector.getAttribute("cpuInfo")[0])
    inspector.getAttribute("cpuInfo")['flags'] = ""
    
    if depth == 1:
        payload.pop('d', None)
        payload.pop('dh', None)
        url = final_url
    
    def make_http_request_to_url(payload):
        try:
            import requests
            response = requests.post(url, json=payload)
            results.append(response.json())
        except Exception as e:
            inspector.addAttribute("call_error", str(e))
            pass
        
    if depth <= 0:
        time.sleep(sleep_time)
        return None
    else:
        threadList = []
        for i in range(breadth):
            thread = threading.Thread(target=make_http_request_to_url, args=(payload,))
            threadList.append(thread)
            thread.start()
        for thread in threadList:
            thread.join()
        inspector.addAttribute("results", results)
        return None
    

request = {
    "depth": 3,
    "breadth": 2,
    "sleep_time": 0.15,
    "url": ds.get_endpoint("skyf_128_af-south-1"),
    "final_url": ds.get_endpoint("skyf_128_af-south-1")
}

result = ds.run(cpu_split, request, "skyf_128_af-south-1", dependencies=["requests"])